In [1]:
#3/9/24 This script takes the DrugbankID -> LNCHI key folder and converts it to a csv file 
import pandas as pd
import numpy as np
import os 

#read in dbankids and lnchi keys as lists
directory = '/Users/james/Documents/Honours/Data/Targets/Drugbank/drugbanktolnchi/'
DBids = []
LNids = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    pathname = directory + file
    with open(pathname, 'r') as file:
        lnchikey = file.read()
    
    DBids.append(filename)
    LNids.append(lnchikey)

In [2]:
#get indexed list of DB Ids and their targets
df = pd.read_csv('/Users/james/Documents/Honours/Data/Targets/Drugbank/Drugbank_PolypeptideTargets.csv')
targets = df['UniProt ID'].values
dbidlist = df['Drug IDs'].values
DBindex = []
for item in dbidlist:
    temp = item.split("; ")
    DBindex.append(temp)

In [10]:
seen = []
for item in DBindex:
    for val in item:
        for bing in val:
            if bing not in seen:
                seen.append(item)
print(len(seen))

156898


In [3]:
#Convert indexed list to inchi keys
inchindex = []
for item in DBindex:
    temp = []
    for name in item:
        loops = 0
        for db in DBids:
            if name == db:
                temp.append(LNids[loops])
            loops += 1
    inchindex.append(temp)

In [4]:
#create one hot dataframe correlating drugs to targets
lns = pd.DataFrame(LNids)
loops = 0
targetframe = []
for item in targets:
    onehot = []
    for key in LNids:
        if key in inchindex[loops]:
            onehot.append(1)
        else:
            onehot.append(0)
    loops += 1
    targetframe.append(onehot)
temp = pd.DataFrame(targetframe)
correlations = temp.transpose()
#correlations.columns = targets

In [5]:
temp = pd.DataFrame(LNids)
temp.columns = ['inchi']

newframe = pd.concat([temp, correlations], axis=1)
test = newframe

In [7]:
#import lnchi keys from datasets
directory = '/Users/james/Documents/Honours/Data/Raws/'
namelist = []
datasets = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        keys = df['InChIKey'].values
        keys = [x for x in keys if str(x) != 'nan']
        datasets.append(df['InChIKey'].values)

INkeys = []
for item in datasets:
    for key in item:
        if key not in INkeys:
            INkeys.append(key)

In [8]:
loops = 0
todrop = []
overlap = []
for item in LNids:
    if item not in INkeys:
        todrop.append(loops)
    else:
        overlap.append(item)
    loops += 1
        

In [10]:
test = newframe.drop(todrop)


In [19]:
newframe

,inchi,0,1,2,3,4,5,6,7,8,...,5340,5341,5342,5343,5344,5345,5346,5347,5348,5349
0,FQWRAVYMZULPNK-BYPYZUCNSA-N,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ZUYUIKKHHBEVHL-GFCCVEGCSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DXGAIOIQACHYRK-QMMMGPOBSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,WUWDLXZGHZSWQZ-WQLSENKSSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,JYXPWUYLZPYOAH-IHWYPQMZSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7059,XUWPJKDMEZSVTP-UOSCCXBLSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7060,KGFLZYXDJDOIEE-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7061,QHMBSVQNZZTUGM-ZWKOTPCHSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7062,UUWJNBOCAPUTBK-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
loops = 0
fileindex = []
for file in os.listdir(directory):
    loops = 0
    filename = os.fsdecode(file)
    if 'csv' in filename:
        fileindex.append(filename)
        pathname = directory + file
        df = pd.read_csv(pathname)
        keys = df['InChIKey'].values
        for item in overlap:
            if item in keys:
                loops += 1
        print(loops, ':', filename)

266 : Cardiotoxicity_Cardiotoxicity-10.csv
192 : Carcinogenicity_Carcinogenicity.csv
1016 : Endocrine Disruption_NR-aromatase.csv
820 : Hepatotoxicity_Hepatotoxicity.csv
47 : Developmental and Reproductive Toxicity_Developmental Toxicity.csv
1162 : Endocrine Disruption_SR-p53.csv
1117 : Endocrine Disruption_NR-AhR.csv
1154 : Endocrine Disruption_NR-AR-LBD.csv
967 : Endocrine Disruption_SR-ARE.csv
266 : Cardiotoxicity_Cardiotoxicity-5.csv
1079 : Endocrine Disruption_NR-ER.csv
266 : Cardiotoxicity_Cardiotoxicity-1.csv
69 : Respiratory Toxicity_Respiratory Toxicity.csv
1207 : Endocrine Disruption_NR-ER-LBD.csv
25 : Developmental and Reproductive Toxicity_Reproductive Toxicity.csv
1073 : Endocrine Disruption_NR-PPAR-gamma.csv
1258 : Endocrine Disruption_NR-AR.csv
390 : Mutagenicity_Ames Mutagenicity.csv
266 : Cardiotoxicity_Cardiotoxicity-30.csv
1211 : Endocrine Disruption_SR-ATAD5.csv
1116 : Endocrine Disruption_SR-HSE.csv
988 : Endocrine Disruption_SR-MMP.csv


In [ ]:
datasets = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        datasets.append(df)

finalsets = []
for df in datasets:
    mutids = df['InChIKey'].values
    toxvals = df['Toxicity Value'].values

    todrop = []
    loops = 0
    for item in mutids:
        if item not in overlap:
            todrop.append(loops)
        loops += 1

    temp = df.drop(todrop)
    
    validids = test['inchi'].values
    loops = 0
    nrarids = []
    for item in validids:
        if item not in mutids:
            nrarids.append(loops)
        loops+=1

    nrar = test.reset_index()
    nrar = nrar.drop(nrarids)
    
    nrartoxvals = []
    loops = 0
    for item in nrar['inchi']:
        loops = 0
        seen = []
        for key in mutids:
            if item == key and key not in seen:
                nrartoxvals.append(toxvals[loops])
            seen.append(key)
            loops += 1     
            
    test = nrar
    test.insert(2, 'Toxicity_Value', nrartoxvals)
    df = nrar.drop(['index'], axis = 1)
    df = nrar
    df = df.drop(['index'], axis = 1)
    finalsets.append(df)

In [12]:
df = pd.read_csv('/Users/james/Documents/Honours/Data/Raws/Endocrine Disruption_NR-AR.csv')
mutids = df['InChIKey'].values
toxvals = df['Toxicity Value'].values

todrop = []
loops = 0
for item in mutids:
    if item not in overlap:
        todrop.append(loops)
    loops += 1

temp = df.drop(todrop)

In [13]:
validids = test['inchi'].values
loops = 0
nrarids = []
for item in validids:
    if item not in mutids:
        nrarids.append(loops)
    loops+=1

In [14]:
len(nrarids)
nrar = test.reset_index()
nrar = nrar.drop(nrarids)

In [15]:
nrartoxvals = []
loops = 0
for item in nrar['inchi']:
    loops = 0
    seen = []
    for key in mutids:
        if item == key and key not in seen:
            nrartoxvals.append(toxvals[loops])
        seen.append(key)
        loops += 1     

In [16]:
test = nrar
test.insert(2, 'Toxicity_Value', nrartoxvals)
df = nrar.drop(['index'], axis = 1)
df = nrar
df = df.drop(['index'], axis = 1)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math
df = df.reset_index()
df = df.drop(['index'],axis = 1)
df = setbalance(df)
mtry = 5
train, test = train_test_split(df, test_size=0.2, random_state=81)
xtrain = train.iloc[:,2:]
ytrain = train['Toxicity_Value'].values
xtest = test.iloc[:,2:]
ytest = test['Toxicity_Value'].values

rf = RandomForestClassifier( criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=81, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
model= rf.fit(xtrain, ytrain)
preds = model.predict(xtest)
results = metriccalc(preds, ytest)
print('validation metrics of:')
print('positives in data', results[0])
print('negatives in data', results[1])
print('net accuracy =', results[6])
print('mcc =',results[12])

validation metrics of:
positives in data 23
negatives in data 28
net accuracy = 0.7254901960784313
mcc = 0.4458225700602822
